# Reindeer Maze

In [13]:
from queue import PriorityQueue
from colorama import Back 

EMPTY = "."
BARRIER = "#"
START = "S"
END = "E"

class Node():
    position = None # (x, y)
    value = None # character
    neighbors = None # [(Node(), >),..]
    direction = None # ^, >, v, <

    def __init__(self, position, value):
        self.position = position
        self.value = value
        self.neighbors = []

    def get_position(self):
        return self.position

    def is_barrier(self):
        return self.value == BARRIER

    def is_start(self):
        return self.value == START

    def is_end(self):
        return self.value == END

    def make_path(self):
        self.value = Back.YELLOW + self.value + Back.RESET

    def update_neighbors(self, grid):
        self.neighbors = []

        x, y = self.position
        max_y = len(grid)
        max_x = len(grid[0])

        if y < max_y - 1 and not grid[y + 1][x].is_barrier():
            self.neighbors.append((grid[y + 1][x], "v"))

        if y > 0 and not grid[y - 1][x].is_barrier():
            self.neighbors.append((grid[y - 1][x], "^"))

        if x < max_x - 1 and not grid[y][x + 1].is_barrier():
            self.neighbors.append((grid[y][x + 1], ">"))

        if x > 0 and not grid[y][x - 1].is_barrier():
            self.neighbors.append((grid[y][x - 1], "<"))
    
    def __str__(self):
        return f"{self.position} {self.value}"

    def __repr__(self):
        return str(self)

def get_matrix(area: str):
    return list(map(lambda line: list(line), area.split("\n")))

def render_matrix(matrix):
    return "\n".join(["".join(line) for line in matrix])

def make_grid(matrix):
    grid = []
    for y in range(len(matrix)):
        grid.append([])
        for x in range(len(matrix[y])):
            value =  matrix[y][x]
            spot = Node((x, y), value)
            grid[y].append(spot)
    return grid

def draw_grid(grid):
    return "\n".join(["".join([node.value for node in row]) for row in grid])

def grid_update_neighbors(grid):
    for row in grid:
        for node in row:
            node.update_neighbors(grid)

# Manhattan distance
def h(current, finish):
    cx, cy = current
    fx, fy = finish
    return abs(cx - fx) + abs(cy - fx)

def reconstruct_path(came_from, current):
    current.make_path()
    while current in came_from:
        current = came_from[current]
        current.make_path()

def get_shortest_path(came_from, current):
    path = [current.get_position()]
    while current in came_from:
        current = came_from[current]
        path.append(current.get_position())
    return path


def astar(grid, start_node, end_node):
    count = 0
    open_set = PriorityQueue()
    open_set.put((0, count, start_node, start_node.direction))
    came_from = {}
    g_score = {node: float("inf") for row in grid for node in row}
    g_score[start_node] = 0
    f_score = {node: float("inf") for row in grid for node in row}
    f_score[start_node] = h(start_node.get_position(), end_node.get_position())

    open_set_hash = {start_node}

    while not open_set.empty():
        current_node, current_direction = open_set.get()[2:]
        open_set_hash.remove(current_node)

        if current_node == end_node:
            reconstruct_path(came_from, end_node)
            return get_shortest_path(came_from, end_node)

        for neighbor, neighbor_direction in current_node.neighbors:
            score = 1

            if current_direction != neighbor_direction:
                score = 1001

            temp_g_score = g_score[current_node] + score

            if temp_g_score < g_score[neighbor]:
                came_from[neighbor] = current_node
                g_score[neighbor] = temp_g_score
                f_score[neighbor] = temp_g_score + h(neighbor.get_position(), end_node.get_position())
                if neighbor not in open_set_hash:
                    count += 1
                    open_set.put((f_score[neighbor], count, neighbor, neighbor_direction))
                    open_set_hash.add(neighbor)
    return None

def get_score(start_node, end_node, path):
        seen = [start_node.get_position()]
        current_node = start_node
        direction = current_node.direction
        score = 0

        while current_node != end_node:
            for neighbor, neighbor_direction in current_node.neighbors:
                if neighbor.get_position() not in path or neighbor.get_position() in seen:
                    continue
                
                if direction == neighbor_direction:
                    score += 1
                    current_node = neighbor
                    seen.append(neighbor.get_position())
                    break
                    
                score += 1001
                current_node = neighbor
                direction = neighbor_direction
                seen.append(neighbor.get_position())

        return score

def get_position(matrix, element):
    for y in range(len(matrix)):
        try:
            x = matrix[y].index(element)
            return (x, y)
        except ValueError:
            continue

with open("test.txt", "r") as f:
    area = f.read().strip()

matrix = get_matrix(area)

# print(render_matrix(matrix))
# print()
grid = make_grid(matrix)

sx, sy = get_position(matrix, START)
ex, ey = get_position(matrix, END)

start_node = grid[sy][sx]
end_node = grid[ey][ex]

start_node.direction = ">"

grid_update_neighbors(grid)

shortest_path = astar(grid, start_node, end_node)

score = get_score(start_node, end_node, shortest_path)

print(f"{score=}")
print(draw_grid(grid))


score=7036
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############


# Attempts

- `89472` That's not the right answer; your answer is too high.

## References

- [A* Pathfinding Visualization Tutorial](https://www.youtube.com/watch?v=JtiK0DOeI4A)
- [Difference and advantages between dijkstra & A star](https://stackoverflow.com/questions/13031462/difference-and-advantages-between-dijkstra-a-star)